In [2]:
import numpy
from src.models.conv_model import build_model as build_conv_model
import tensorflow as tf
from src.evaluator.evaluator import Evaluator
from src.models.conv_model import correlation_coefficient_loss, pearson_r
import math
from src.data_loader.RNASeqLoader import RNASeqDataGenerator 
dependencies = {
    'correlation_coefficient_loss': correlation_coefficient_loss,
    'pearson_r': pearson_r
}
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
train_generator = RNASeqDataGenerator("data/processed/cDNA-CBE/train_data.hdf5", 512)
validation_generator = RNASeqDataGenerator("data/processed/cDNA-CBE/validation_data.hdf5", 512)

model = tf.keras.models.load_model('models/cDNA-CBE/op.h5', custom_objects=dependencies)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
print(model.summary())
checkpoint_filepath = "models/cDNA-CBE/opmodel-{epoch:02d}-{val_loss:.4f}.h5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    #monitor='val_pearson_r',
    #mode='max',
    save_best_only=False)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=4, min_lr=0.00001)

history = model.fit(x=train_generator, epochs=30, validation_data=validation_generator, callbacks=[model_checkpoint_callback], use_multiprocessing=True, workers=5)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 101, 64)           4160      
_________________________________________________________________
batch_normalization (BatchNo (None, 101, 64)           256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 101, 32)           32800     
_________________________________________________________________
batch_normalization_1 (Batch (None, 101, 32)           128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 101, 16)           4112      
_________________________________________________________________
batch_normalization_2 (Batch (None, 101, 16)           64        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 26, 16)            0

Process Keras_worker_ForkPoolWorker-99:
Process Keras_worker_ForkPoolWorker-95:
Process Keras_worker_ForkPoolWorker-94:
Process Keras_worker_ForkPoolWorker-97:
Process Keras_worker_ForkPoolWorker-100:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/tf1.15.2/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/tf1.15.2/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/envs/tf1.15.2/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/miniconda3/envs/tf1.15.2/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 571, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/home/harsh/src/data_loader/RNASeqLoader.py", line 46, in __getitem__
    return self.__data_generation(indexes)
  File "/home/harsh/src/data_loader/RNASeqLoader.py", line 59, i

KeyboardInterrupt: 